# Math 134C Week 9

In [2]:
import math
import numpy as np
from scipy.stats import norm
from scipy.stats.mstats import gmean

def black_scholes_call(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = S_t * np.exp(-delta * dt) * norm.cdf(d1) - K * np.exp(-r * dt) * norm.cdf(d2)
    return call_price

def black_scholes_put(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    call_price = - S_t * np.exp(-delta * dt) * norm.cdf(-d1) + K * np.exp(-r * dt) * norm.cdf(-d2)
    return call_price

def call_Delta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))  
    return np.exp(-delta * dt) * norm.cdf(d1)

def put_Delta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))  
    return -np.exp(-delta * dt) * norm.cdf(-d1)

def Gamma(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    return np.exp(-delta * dt) * norm.pdf(d1)/(S_t * sigma * np.sqrt(dt)) 

def call_Theta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    return delta * S_t * np.exp(-delta * dt)*norm.cdf(d1) - r * K * np.exp(-r*dt)*norm.cdf(d2)-(K * np.exp(-r*dt) * sigma * norm.pdf(d2))/ (2 * np.sqrt(dt)) 

def put_Theta(S_t, K, dt, r, sigma, delta):
    d1 = (np.log(S_t / K) + (r -delta + 0.5 * sigma ** 2) * dt) / (sigma * np.sqrt(dt))
    d2 = d1 - sigma * np.sqrt(dt)
    return call_Theta(S_t, K, dt, r, sigma, delta) + r * K * np.exp(-r*dt) - delta * S_t * np.exp(-delta * dt)

def put_given_call(C_t, S_t, K, r, dt):
    return C_t - S_t + K * math.exp(-r * dt)
def call_given_put(P_t, S_t, K, r, dt):
    return P_t + S_t - K * math.exp(-r * dt)

def one_step_binomial(r, dt, u, d, Vu, Vd, delta = 0):
    q = (math.exp((r - delta) * dt) - d)/(u-d)
    if not (1 > q > 0): 
        print("Arbitrage opportunity exists")
        return -1
    return math.exp(-r * dt)*(q*Vu+(1-q)*Vd)    

**Quiz 8.1**
![](img/week9_1.png)

In [90]:
price = [31, 35, 40, 38, 39, 36]
amean_price = np.mean(price)
gmean_price = gmean(price)
print(f"The arithematic mean of the price is {amean_price:.2f}")
print(f"The geometric mean of the price is {gmean_price:.2f}")
payoff_A = max(0, amean_price - 36)
payoff_B = max(0, gmean_price - 35)
print(f"The value of 100(B-A) is {100*(payoff_B-payoff_A):.2f}")

The arithematic mean of the price is 36.50
The geometric mean of the price is 36.37
The value of 100(B-A) is 87.25


**Quiz 8.2**
![](img/week9_2.png)

In [91]:
u = np.exp((0.03-0.01)*0.5 + 0.3 * np.sqrt(0.5))
d = np.exp((0.03-0.01)*0.5 - 0.3 * np.sqrt(0.5))

S = 40
Su = S * u
Sd = S * d
Suu = Su * u
Sud = Su * d
Sdd = Sd * d

Average_uu = gmean([Su, Suu])
Average_ud = gmean([Su, Sud])
Average_du = gmean([Sd, Sud])
Average_dd = gmean([Sd, Sdd])

P_uu = max(0, Average_uu - Suu)
P_ud = max(0, Average_ud - Sud)
P_du = max(0, Average_du - Sud)
P_dd = max(0, Average_dd - Sdd)

q = (np.exp(0.02 * 0.5) - d)/(u-d)

# One method to compute the price at each node
P_u = np.exp(-0.03 * 0.5) * (q * P_uu + (1-q) * P_ud)
P_d = np.exp(-0.03 * 0.5) * (q * P_du + (1-q) * P_dd)
P = np.exp(-0.03 * 0.5) * (q * P_u + (1-q) * P_d)

# Another method to compute the price directly
# P = np.exp(-0.03) * (q**2 * P_uu + q*(1-q) * P_ud + q*(1-q)*P_du + (1-q)**2*P_dd)
print(f"The price of this option is {P:.1f}")

The price of this option is 1.9


**Quiz 8.3**
![](img/week9_3.png)

**Quiz 8.4**
For this problem, please construct a two-period binomial model
![](img/week9_4.png)

In [93]:
u = np.exp((0.06-0.05) + 0.3)
d = np.exp((0.06-0.05) - 0.3)
S = 100
Su = S * u
Sd = S * d
Suu = S * u * u
Sud = S * u * d
Sdd = S * d * d

Puu = 0
Pud = 0
Pdu = 120 - Sud
Pdd = 120 - Sdd
q = (np.exp(0.01) - d)/(u-d)
Pu = 0
Pd = np.exp(-0.06) * (q * Pdu + (1-q) * Pdd)

P = np.exp(-0.06) * (q * Pu + (1-q) * Pd)
print(f"The price of this option is {P:.2f}")

The price of this option is 22.63


**Quiz 8.5**
![](./img/week9_5.png)

**Quiz 8.6**
![](img/week9_6.png)
You are also given that the volatility of the stock is less than 100%.

In [124]:
delta = 0.61791
r = 0.1
S = 50
T = 0.25
sigma = 0.2

# delta_guessed = call_Delta(S, S, T, r, sigma, 0)
# print(delta_guessed)

dS = 0.52
C_old = black_scholes_call(S, S, T, r, sigma, 0)
C_new = black_scholes_call(S+dS, S, T-1/365, r, sigma, 0)

profit = -(C_new - C_old) + dS * delta - (-C_old + delta * S) * (np.exp(r/365) - 1)
print("The one day profit is", profit, "if the stock price changes by", dS)

The one day profit is 0.0005801055773632961 if the stock price changes by 0.52


**Quiz 8.7**
![](img/week9_7.png)

**Quiz 8.8**
![](img/week9_8.png)

**Quiz 8.9**
![](img/week9_9.png)

**Quiz 8.10**
![](img/week9_10.png)

# Compound Options


# Monte Carlo Methods

In [ ]:
import matplotlib.pyplot as plt

def stock_price_simulation(T, N, r, delta, sigma, S0):
    dt = float(T) / N
    t = np.linspace(0, T, N+1)
    S = np.zeros(N+1)
    S[0] = S0
    for i in range(1, N+1):
        Z = np.random.normal(0, 1)
        S[i] = S[i-1] * np.exp((r - delta - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return t, S

T = 1
N = 365
r = 0.0535
delta = 0
sigma = 0.34
S0 = 41.75

number_of_trials = 10000

payoff = 0
for i in range(number_of_trials):
    t, S = stock_price_simulation(T, N, r, delta, sigma, S0)    
    payoff += max(0, 55-S[-1])
    
print(payoff/number_of_trials * np.exp(-0.0535))


In [ ]:
black_scholes_put(41.75, 55, 1, 0.0535, 0.34, 0)